In [4]:
import requests
import base64
import datetime
from urllib.parse import urlencode

In [5]:
client_id = '18bf732ea7ab485e91a45dfe7b75d5ec'
client_secret = '61d0f5271ab34e7aa519f1bebd81d3c9'

In [34]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception("A query is required.")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() ==  "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [35]:
spotify = SpotifyAPI(client_id, client_secret)

In [36]:
spotify.search({"track": "Time", "artist": "Hans"}, search_type='track')

q=track%3ATime+artist%3AHans&type=track


,
     'PY',
     'QA',
     'RO',
     'RS',
     'RU',
     'SA',
     'SE',
     'SG',
     'SI',
     'SK',
     'SV',
     'TH',
     'TN',
     'TR',
     'TW',
     'UA',
     'US',
     'UY',
     'VN',
     'XK',
     'ZA'],
    'disc_number': 1,
    'duration_ms': 170016,
    'explicit': False,
    'external_ids': {'isrc': 'USWB10807407'},
    'external_urls': {'spotify': 'https://open.spotify.com/track/5HTfALgK80g3MMddAKfkYd'},
    'href': 'https://api.spotify.com/v1/tracks/5HTfALgK80g3MMddAKfkYd',
    'id': '5HTfALgK80g3MMddAKfkYd',
    'is_local': False,
    'name': 'Decent Men in an Indecent Time',
    'popularity': 22,
    'preview_url': 'https://p.scdn.co/mp3-preview/b720161f46e1575886389a5e784c612b94203b42?cid=18bf732ea7ab485e91a45dfe7b75d5ec',
    'track_number': 18,
    'type': 'track',
    'uri': 'spotify:track:5HTfALgK80g3MMddAKfkYd'},
   {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7IdgUqOoYzcoL

In [ ]:
# spotify.get_artist('2xiIXseIJcq3nG7C8fHeBj')

In [ ]:
# spotify.get_album("4ZDwEvXIKxWdwhAQK8ndBU")

In [38]:
spotify.search(query="Danger", operator="NOT", operator_query="Zone", search_type='track')

q=Danger+Zone&type=track


138003,
    'explicit': True,
    'external_ids': {'isrc': 'DEQ121855131'},
    'external_urls': {'spotify': 'https://open.spotify.com/track/1p2CLrC6aBMr5EtokUqdQd'},
    'href': 'https://api.spotify.com/v1/tracks/1p2CLrC6aBMr5EtokUqdQd',
    'id': '1p2CLrC6aBMr5EtokUqdQd',
    'is_local': False,
    'name': 'Danger Zone Blues - raw',
    'popularity': 31,
    'preview_url': 'https://p.scdn.co/mp3-preview/475c17ad965ab01793337f622a99febe8341a989?cid=18bf732ea7ab485e91a45dfe7b75d5ec',
    'track_number': 18,
    'type': 'track',
    'uri': 'spotify:track:1p2CLrC6aBMr5EtokUqdQd'},
   {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/65T6BIrUTcgylCYTXqVxvR'},
       'href': 'https://api.spotify.com/v1/artists/65T6BIrUTcgylCYTXqVxvR',
       'id': '65T6BIrUTcgylCYTXqVxvR',
       'name': 'Baker Ya Maker',
       'type': 'artist',
       'uri': 'spotify:artist:65T6BIrUTcgylCYTXqVxvR'}],
     'available_markets': ['AD',
      'A

In [39]:
spotify.search(query="Danger Zone", search_type='track')

q=Danger+Zone&type=track


138003,
    'explicit': True,
    'external_ids': {'isrc': 'DEQ121855131'},
    'external_urls': {'spotify': 'https://open.spotify.com/track/1p2CLrC6aBMr5EtokUqdQd'},
    'href': 'https://api.spotify.com/v1/tracks/1p2CLrC6aBMr5EtokUqdQd',
    'id': '1p2CLrC6aBMr5EtokUqdQd',
    'is_local': False,
    'name': 'Danger Zone Blues - raw',
    'popularity': 31,
    'preview_url': 'https://p.scdn.co/mp3-preview/475c17ad965ab01793337f622a99febe8341a989?cid=18bf732ea7ab485e91a45dfe7b75d5ec',
    'track_number': 18,
    'type': 'track',
    'uri': 'spotify:track:1p2CLrC6aBMr5EtokUqdQd'},
   {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/65T6BIrUTcgylCYTXqVxvR'},
       'href': 'https://api.spotify.com/v1/artists/65T6BIrUTcgylCYTXqVxvR',
       'id': '65T6BIrUTcgylCYTXqVxvR',
       'name': 'Baker Ya Maker',
       'type': 'artist',
       'uri': 'spotify:artist:65T6BIrUTcgylCYTXqVxvR'}],
     'available_markets': ['AD',
      'A